In [2]:
# Import Dependencies
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [2]:
# Import DataFrame 
main_df = pd.read_csv('Rates_MO.csv')
main_df.columns

Index(['Unnamed: 0', 'observation_date', 'C&I_DELNQ', 'CCARD_CO',
       'CCARD_DELNQ', 'CORP_DEBT_NET_WORTH', 'CORP_SAVINGS_LEVEL', 'CRE_CO',
       'CRE_DELNQ', 'GDP', 'Homeowner_Vacancy_rate', 'Household_DBT_Inc',
       'Mortgage_CO', 'Mortgage_DELNQ', 'Rental_Vacancy_Rate',
       'Consumer_Confidence', 'FEDFUNDS', 'Manufacturing_Confidence',
       'SAVINGS_RATE_MO', 'UNRATE', 'C&I_CO'],
      dtype='object')

In [3]:
# Create credit card dataset
ccard_df = main_df[['observation_date', 'CCARD_CO', 'CCARD_DELNQ', 'GDP', 'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS', 'SAVINGS_RATE_MO', 'UNRATE']]

In [4]:
ccard_df.dtypes

observation_date        object
CCARD_CO               float64
CCARD_DELNQ            float64
GDP                    float64
Household_DBT_Inc      float64
Consumer_Confidence    float64
FEDFUNDS               float64
SAVINGS_RATE_MO        float64
UNRATE                 float64
dtype: object

In [5]:
ccard_bin_df = ccard_df.copy()

In [6]:
#ccard_df['CCARD_CO']=ccard_df['CCARD_CO'].astype(int)

In [7]:
# Find Minimum and Maximum values to determine number of bins
a=ccard_df['CCARD_CO'].max()
b=ccard_df['CCARD_CO'].min()
c=ccard_df['CCARD_CO'].count()
print(a)
print(b)
print(c)

10.51
1.66
390


In [1]:
# Binning the data for classification Question: 
# Should we be using pd cut to get more bins? 
ccard_bin_df["CCARD_CO_BIN"] = pd.qcut(ccard_df['CCARD_CO'],4, labels= [1, 2, 3, 4])
ccard_bin_df["CCARD_DELNQ_BIN"] = pd.qcut(ccard_df['CCARD_DELNQ'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["GDP_BIN"] = pd.qcut(ccard_df['GDP'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["Household_DBT_Inc_BIN"] = pd.qcut(ccard_df['Household_DBT_Inc'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["Consumer_Confidence_BIN"] = pd.qcut(ccard_df['Consumer_Confidence'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["FEDFUNDS_BIN"] = pd.qcut(ccard_df['FEDFUNDS'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["SAVINGS_RATE_MO_BIN"] = pd.qcut(ccard_df['SAVINGS_RATE_MO'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["UNRATE_BIN"] = pd.qcut(ccard_df['UNRATE'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])


NameError: name 'pd' is not defined

In [9]:
ccard_bin_df.head()

,observation_date,CCARD_CO,CCARD_DELNQ,GDP,Household_DBT_Inc,Consumer_Confidence,FEDFUNDS,SAVINGS_RATE_MO,UNRATE,CCARD_CO_BIN,CCARD_DELNQ_BIN,GDP_BIN,Household_DBT_Inc_BIN,Consumer_Confidence_BIN,FEDFUNDS_BIN,SAVINGS_RATE_MO_BIN,UNRATE_BIN
0,1991-01-01,4.16,5.26,-1.9,11.578032,66.8,6.91,9.4,6.4,medium-high,high,low,medium-high,low,high,high,medium-high
1,1991-02-01,4.16,5.26,-1.9,11.578032,70.4,6.25,9.0,6.6,medium-high,high,low,medium-high,low,high,high,medium-high
2,1991-03-01,4.16,5.26,-1.9,11.578032,87.7,6.12,8.1,6.8,medium-high,high,low,medium-high,medium-low,high,high,high
3,1991-04-01,4.60,5.48,3.2,11.434237,81.8,5.91,8.7,6.7,medium-high,high,medium-high,medium-high,medium-low,high,high,medium-high
4,1991-05-01,4.60,5.48,3.2,11.434237,78.3,5.78,8.5,6.9,medium-high,high,medium-high,medium-high,medium-low,high,high,high


In [10]:
# Seperate the y and X variables
y = ccard_bin_df["CCARD_CO_BIN"]
X_bin = ccard_bin_df.drop(columns=["CCARD_CO", "observation_date", "CCARD_DELNQ", "GDP", "Household_DBT_Inc", "Consumer_Confidence", "FEDFUNDS", "SAVINGS_RATE_MO", "UNRATE", "CCARD_CO_BIN"])
X_bin.columns

Index(['CCARD_CO_BIN', 'CCARD_DELNQ_BIN', 'GDP_BIN', 'Household_DBT_Inc_BIN',
       'Consumer_Confidence_BIN', 'FEDFUNDS_BIN', 'SAVINGS_RATE_MO_BIN',
       'UNRATE_BIN'],
      dtype='object')

In [11]:
# Turn data into dummies
X_bin =pd.get_dummies(X_bin)
X = pd.concat([y, X_bin])
X

,CCARD_CO_BIN_low,CCARD_CO_BIN_medium-low,CCARD_CO_BIN_medium-high,CCARD_CO_BIN_high,CCARD_DELNQ_BIN_low,CCARD_DELNQ_BIN_medium-low,CCARD_DELNQ_BIN_medium-high,CCARD_DELNQ_BIN_high,GDP_BIN_low,GDP_BIN_medium-low,...,FEDFUNDS_BIN_medium-high,FEDFUNDS_BIN_high,SAVINGS_RATE_MO_BIN_low,SAVINGS_RATE_MO_BIN_medium-low,SAVINGS_RATE_MO_BIN_medium-high,SAVINGS_RATE_MO_BIN_high,UNRATE_BIN_low,UNRATE_BIN_medium-low,UNRATE_BIN_medium-high,UNRATE_BIN_high
0,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0
1,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0
2,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,0,1
3,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
4,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1


In [12]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X_bin, y, random_state=78)

In [13]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
len(X_train_scaled)

292

In [15]:
# Create a random forest classifier
cc_model = RandomForestClassifier(n_estimators=100, random_state=78)

In [16]:
# Fitting the model
cc_model = cc_model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data
predictions = cc_model.predict(X_test_scaled)

In [18]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual Low", "Actual Medium-Low", "Actual Medium-High", "High"], columns=["Predicted Low", "Predicted Medium-Low", "Predicted Medium-High", "Predicted High"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [19]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Low,Predicted Medium-Low,Predicted Medium-High,Predicted High
Actual Low,25,0,0,0
Actual Medium-Low,0,25,0,0
Actual Medium-High,0,0,24,0
High,0,0,0,24


Accuracy Score : 1.0
Classification Report
              precision    recall  f1-score   support

        high       1.00      1.00      1.00        25
         low       1.00      1.00      1.00        25
 medium-high       1.00      1.00      1.00        24
  medium-low       1.00      1.00      1.00        24

    accuracy                           1.00        98
   macro avg       1.00      1.00      1.00        98
weighted avg       1.00      1.00      1.00        98



# Part 2: Logistic Regression Model

In [20]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [21]:
# Train the data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [22]:
# Predict outcomes for test data set
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
384,low,low
206,medium-high,medium-high
132,high,high
226,high,high
223,high,high
...,...,...
300,low,low
293,low,low
71,medium-high,medium-high
370,low,low


In [23]:
from sklearn.metrics import accuracy_score
# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

1.0

# Part 3: Deep Learning First Attempt 

In [24]:
ccard_df.columns

Index(['observation_date', 'CCARD_CO', 'CCARD_DELNQ', 'GDP',
       'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS',
       'SAVINGS_RATE_MO', 'UNRATE'],
      dtype='object')

In [25]:
# Seperate the y and X variables
# y = ccard_df["CCARD_CO"]
# X = ccard_df.drop(columns=["CCARD_CO", "observation_date"])
# X.columns

Index(['CCARD_DELNQ', 'GDP', 'Household_DBT_Inc', 'Consumer_Confidence',
       'FEDFUNDS', 'SAVINGS_RATE_MO', 'UNRATE'],
      dtype='object')

In [26]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [27]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
print(X_train_scaled)

[[-1.11044114 -0.09503839 -1.02558144 ...  0.89583826 -0.44699742
  -1.24527544]
 [ 0.47017089  0.81483737  0.21679844 ...  1.19237309 -0.19544696
  -0.10539949]
 [ 0.75365022 -0.39173701 -0.52701132 ...  0.21152713  0.84219871
   0.69251368]
 ...
 [ 0.7106988   0.79505747  0.45821952 ...  1.18324894 -0.66710408
  -0.96030645]
 [-0.37167682  0.55769857 -0.49785455 ...  0.64036211  0.4334292
   0.1795695 ]
 [ 0.25541382 -0.03569867  1.67834979 ...  1.20605931 -1.07587359
  -0.67533746]]


In [42]:
import tensorflow as tf

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled)
hidden_nodes_layers1 = 100
hidden_nodes_layers2 = 100
hidden_nodes_layers3 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=7, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               800       
                                                                 
 dense_13 (Dense)            (None, 100)               10100     
                                                                 
 dense_14 (Dense)            (None, 100)               10100     
                                                                 
 dense_15 (Dense)            (None, 1)                 101       
                                                                 
Total params: 21101 (82.43 KB)
Trainable params: 21101 (82.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [44]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
10/10 [==============================] - 0s 730us/step - loss: -0.4348 - accuracy: 0.0000e+00
Epoch 2/100
10/10 [==============================] - 0s 573us/step - loss: -5.8055 - accuracy: 0.0000e+00
Epoch 3/100
10/10 [==============================] - 0s 658us/step - loss: -15.5558 - accuracy: 0.0000e+00
Epoch 4/100
10/10 [==============================] - 0s 592us/step - loss: -36.3000 - accuracy: 0.0000e+00
Epoch 5/100
10/10 [==============================] - 0s 650us/step - loss: -78.9957 - accuracy: 0.0000e+00
Epoch 6/100
10/10 [==============================] - 0s 506us/step - loss: -166.1005 - accuracy: 0.0000e+00
Epoch 7/100
10/10 [==============================] - 0s 658us/step - loss: -312.6646 - accuracy: 0.0000e+00
Epoch 8/100
10/10 [==============================] - 0s 598us/step - loss: -573.2451 - accuracy: 0.0000e+00
Epoch 9/100
10/10 [==============================] - 0s 619us/step - loss: -979.7945 - accuracy: 0.0000e+00
Epoch 10/100
10/10 [===============

10/10 [==============================] - 0s 581us/step - loss: -16248664.0000 - accuracy: 0.0000e+00
Epoch 75/100
10/10 [==============================] - 0s 663us/step - loss: -17153126.0000 - accuracy: 0.0000e+00
Epoch 76/100
10/10 [==============================] - 0s 580us/step - loss: -18143448.0000 - accuracy: 0.0000e+00
Epoch 77/100
10/10 [==============================] - 0s 807us/step - loss: -19106906.0000 - accuracy: 0.0000e+00
Epoch 78/100
10/10 [==============================] - 0s 617us/step - loss: -20142146.0000 - accuracy: 0.0000e+00
Epoch 79/100
10/10 [==============================] - 0s 698us/step - loss: -21209324.0000 - accuracy: 0.0000e+00
Epoch 80/100
10/10 [==============================] - 0s 578us/step - loss: -22248494.0000 - accuracy: 0.0000e+00
Epoch 81/100
10/10 [==============================] - 0s 990us/step - loss: -23405508.0000 - accuracy: 0.0000e+00
Epoch 82/100
10/10 [==============================] - 0s 723us/step - loss: -24642306.0000 - accuracy

# Deep Learning Second Attempt

In [45]:
# Seperate the y and X variables
y = ccard_bin_df["CCARD_CO_BIN"]
X = ccard_bin_df.drop(columns=["CCARD_CO", "observation_date", "CCARD_DELNQ", "GDP", "Household_DBT_Inc", "Consumer_Confidence", "FEDFUNDS", "SAVINGS_RATE_MO", "UNRATE"])
X.columns

Index(['CCARD_CO_BIN', 'CCARD_DELNQ_BIN', 'GDP_BIN', 'Household_DBT_Inc_BIN',
       'Consumer_Confidence_BIN', 'FEDFUNDS_BIN', 'SAVINGS_RATE_MO_BIN',
       'UNRATE_BIN'],
      dtype='object')

In [46]:
# Turn data into dummies
X=pd.get_dummies(X)

X.head()

,CCARD_CO_BIN_low,CCARD_CO_BIN_medium-low,CCARD_CO_BIN_medium-high,CCARD_CO_BIN_high,CCARD_DELNQ_BIN_low,CCARD_DELNQ_BIN_medium-low,CCARD_DELNQ_BIN_medium-high,CCARD_DELNQ_BIN_high,GDP_BIN_low,GDP_BIN_medium-low,...,FEDFUNDS_BIN_medium-high,FEDFUNDS_BIN_high,SAVINGS_RATE_MO_BIN_low,SAVINGS_RATE_MO_BIN_medium-low,SAVINGS_RATE_MO_BIN_medium-high,SAVINGS_RATE_MO_BIN_high,UNRATE_BIN_low,UNRATE_BIN_medium-low,UNRATE_BIN_medium-high,UNRATE_BIN_high
0,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0
1,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0
2,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,0,1
3,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
4,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1


In [56]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [57]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [58]:
print(X_train_scaled)

[[ 1.71637582 -0.57207755 -0.58789635 ... -0.57735027 -0.59317101
  -0.56152944]
 [-0.58262298 -0.57207755  1.70098011 ... -0.57735027  1.68585446
  -0.56152944]
 [-0.58262298  1.74801475 -0.58789635 ... -0.57735027 -0.59317101
   1.78085052]
 ...
 [-0.58262298 -0.57207755  1.70098011 ... -0.57735027 -0.59317101
  -0.56152944]
 [ 1.71637582 -0.57207755 -0.58789635 ... -0.57735027  1.68585446
  -0.56152944]
 [-0.58262298  1.74801475 -0.58789635 ...  1.73205081 -0.59317101
  -0.56152944]]


In [62]:
X_train_scaled.dtype

dtype('float64')

In [52]:
import tensorflow as tf

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled)
hidden_nodes_layers1 = 100
hidden_nodes_layers2 = 100
hidden_nodes_layers3 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=32, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 100)               3300      
                                                                 
 dense_21 (Dense)            (None, 100)               10100     
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dense_23 (Dense)            (None, 1)                 101       
                                                                 
Total params: 23601 (92.19 KB)
Trainable params: 23601 (92.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [54]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


2023-11-10 16:35:17.521123: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/Users/ryak/anaconda3/envs/dev/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  Fil